In [1]:
from re import finditer, search, match
import pandas as pd

In [2]:
fileCA = open("source/sample-ca.txt", "r")
fileUSA = open("source/united_states_profiles.txt", "r")
fileSI = open("source/singapore_profiles.txt", "r")
fileIL = open("source/sample-il.txt", "r")
lines = fileUSA.readlines() + fileSI.readlines() + fileCA.readlines() + fileIL.readlines()
fileCA.close()
fileUSA.close()
fileSI.close()
fileIL.close()

In [3]:
reg_val = ":.+"
reg = "\"[^:,]+\": (((\[\]|\[.*?(}|\")\])(,|}))|((\"\")|(\".+?\"),)|.*?,)"

columns = [i.group().split(':')[0].strip('"') for i in finditer(reg,lines[0])]
valeurs = [[search(reg_val,i.group()).group().strip(': ",}') for i in finditer(reg,line)] for line in lines]

df = pd.DataFrame(valeurs, columns = columns)

In [4]:
atom_columns = ['public_identifier',
 'profile_pic_url',
 'background_cover_image_url',
 'first_name',
 'last_name',
 'full_name',
 'occupation',
 'headline',
 'summary',
 'country',
 'country_full_name',
 'city',
 'state',
 'connections']

df_profils = df[atom_columns]
df_profils.to_csv('destination/profils.csv', sep=";", index=False)

In [5]:
reg_col = '[^:,]+": ({.+?},|(".+?[^\\\]")|.*?(,|}))'
reg_date = "[0-9]{1,4}"
reg_sep = "{([^{}]*{[^{}]*}[^{}]*)*}|{.+?}"

rep_col = lambda s : s.strip('{').strip(' ').strip('"') 
sep = lambda column,i_line : [unique.group() for unique in finditer(reg_sep,df[column][i_line])]
pref_col = lambda column : ['public_identifier','num_'+column]
sub = lambda column,i_line :  pref_col(column) + [rep_col(i.group().split(': ')[0]) for i in finditer(reg_col,sep(column,i_line)[0])]
profil = lambda i_line : df['public_identifier'][i_line]

def sub_cols(column):
    test_line = 0
    while (sep(column,test_line) == []) :
        test_line +=1
    return sub(column,test_line)

In [6]:
sub_val = lambda column,i_line,i_col : [profil(i_line), i_col+1]+['-'.join([date.group().zfill(2) for date in finditer(reg_date, search(reg_val,val.group()).group().strip(': ",'))]) if '{' in val.group() else (search(reg_val,val.group()).group().strip(': ",}')) for val in finditer(reg_col,sep(column,i_line)[i_col])]
sub_vals = lambda column : [sub_val(column,i_line,i_col) for i_line in range(len(lines)) for i_col in range(len(sep(column,i_line)))]

non_atom_cols = ['experiences',
 'education',
 #'languages',
 'accomplishment_organisations',
 'accomplishment_publications',
 'accomplishment_honors_awards',
 'accomplishment_patents',
 'accomplishment_courses',
 'accomplishment_projects',
 'accomplishment_test_scores',
 'volunteer_work',
 'certifications',
 #'people_also_viewed',
 'recommendations',
 'activities',
 'similarly_named_profiles',
 'articles',
 #'groups'
                ]

dfs = [pd.DataFrame(sub_vals(non_atom_col), columns = sub_cols(non_atom_col)) for non_atom_col in non_atom_cols]
[dfs[i_df].to_csv('destination/'+non_atom_cols[i_df]+'.csv', sep=";", index=False) for i_df in range(len(non_atom_cols))]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
sub_cols_languages = pref_col('languages') + ['language']
rep_lang = lambda s : s.strip('][').replace(' ','').replace('"','').split(',')
languages = lambda i_line : rep_lang(df['languages'][i_line])
sub_vals_languages = [[profil(i_line),num_language+1,languages(i_line)[num_language]] for i_line in range(len(lines)) for num_language in range(len(languages(i_line))) if languages(i_line) !=['']]

df_languages = pd.DataFrame(sub_vals_languages, columns = sub_cols_languages)
df_languages.to_csv('destination/languages.csv', sep=";", index=False)

In [9]:
reg_sep_rec = '".+?"(,|\])'
sub_cols_recommendations = pref_col('recommendations') + ['recommendation']
sep_rec = lambda i_line : [unique.group() for unique in finditer(reg_sep_rec,df['recommendations'][i_line])]
rep_rec = lambda s : s.replace('\\"','"').replace('",','').replace('"','').replace('  ','').replace(']','')
sub_val_recommendation = lambda i_line,i_col : [profil(i_line),i_col+1,rep_rec(sep_rec(i_line)[i_col])]
sub_vals_recommendations = [sub_val_recommendation(i_line,i_col) for i_line in range(len(lines)) for i_col in range(len(sep_rec(i_line)))]

df_recommendations = pd.DataFrame(sub_vals_recommendations, columns = sub_cols_recommendations)
df_recommendations.to_csv('destination/recommendations.csv', sep=";", index=False)

In [10]:
sub_vals_groups = [sub_val('groups',i_line,i_col) for i_line in range(len(lines[:20000])) for i_col in range(len(sep('groups',i_line)))]
df_groups = pd.DataFrame(sub_vals_groups, columns = sub_cols('groups'))
df_groups.to_csv('destination/groups.csv', sep=";", index=False)

In [11]:
sub_vals_people_also_viewed1 = [sub_val('people_also_viewed',i_line,i_col) for i_line in range(len(lines[:20000])) for i_col in range(len(sep('people_also_viewed',i_line)))]
sub_vals_people_also_viewed2 = [sub_val('people_also_viewed',i_line,i_col) for i_line in range(len(lines[20000:])) for i_col in range(len(sep('people_also_viewed',i_line)))]
sub_vals_people_also_viewed = sub_vals_people_also_viewed1+sub_vals_people_also_viewed2

df_people_also_viewed = pd.DataFrame(sub_vals_people_also_viewed, columns = sub_cols('people_also_viewed'))
df_people_also_viewed.to_csv('destination/people_also_viewed.csv', sep=";", index=False)

In [ ]:
""" DEBOGAGE ATOMIQUE """
#[search(reg_val,i.group()).group().strip(': ",}') for i in finditer(reg,lines[474])][:12]
# df['state'].iloc[474]
# lines[474]
# df['connections'].value_counts()
# v=''; df[(df['connections'] == v)]

In [14]:
""" DEBOGAGE NON ATOMIQUE """

# sub_vals('accomplishment_honors_awards')
# sub_cols('accomplishment_honors_awards')
# df[['accomplishment_honors_awards', 'public_identifier']].head(15)
# df[df['accomplishment_honors_awards']!='[]'].accomplishment_honors_awards
# df['accomplishment_honors_awards'][37]
# sub_val('accomplishment_honors_awards',37,0)
# [unique.group() for unique in finditer(reg_sep,df['accomplishment_honors_awards'][37])]


' DEBOGAGE NON ATOMIQUE '